# 🦠 COVID-19 SQL EDA with Jupyter and MySQL

This notebook connects to a MySQL database using `mysql-connector-python` and performs exploratory analysis on COVID-19 data.

In [ ]:
!pip install mysql-connector-python

In [ ]:

import mysql.connector
import pandas as pd

# Establish database connection (update with your credentials)
conn = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password",
    database="covid_eda"
)

cursor = conn.cursor()


In [ ]:

def run_query(query):
    cursor.execute(query)
    columns = [col[0] for col in cursor.description]
    return pd.DataFrame(cursor.fetchall(), columns=columns)


## 🔹 Total Cases and Deaths by Country

In [ ]:

query = '''
SELECT location, MAX(total_cases) AS total_cases, MAX(total_deaths) AS total_deaths
FROM covid_deaths
WHERE continent IS NOT NULL
GROUP BY location
ORDER BY total_cases DESC;
'''
run_query(query)


## 🔹 Global Highest Daily New Cases

In [ ]:

query = '''
SELECT location, date, new_cases
FROM covid_deaths
WHERE new_cases IS NOT NULL
ORDER BY new_cases DESC
LIMIT 1;
'''
run_query(query)


## 🔹 Death Rate by Country

In [ ]:

query = '''
SELECT location, MAX(total_deaths) / MAX(total_cases) * 100 AS death_rate_percent
FROM covid_deaths
WHERE continent IS NOT NULL
GROUP BY location
ORDER BY death_rate_percent DESC;
'''
run_query(query)


## 🔹 7-Day Average of New Cases

In [ ]:

query = '''
SELECT location, date,
       AVG(new_cases) OVER (PARTITION BY location ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS avg_new_cases_7_days
FROM covid_deaths
WHERE continent IS NOT NULL;
'''
run_query(query)


## 🔹 Top 10 Countries by Cases Per Million

In [ ]:

query = '''
SELECT location, MAX(total_cases_per_million) AS cases_per_million
FROM covid_deaths
WHERE continent IS NOT NULL
GROUP BY location
ORDER BY cases_per_million DESC
LIMIT 10;
'''
run_query(query)


## 🔹 Highest ICU Patients in a Single Day

In [ ]:

query = '''
SELECT date, location, icu_patients
FROM covid_deaths
WHERE icu_patients IS NOT NULL
ORDER BY icu_patients DESC
LIMIT 1;
'''
run_query(query)


In [ ]:
cursor.close()
conn.close()